## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-20-19-12-11 +0000,nyt,Greenland’s Leader Says He Cannot Rule Out an ...,https://www.nytimes.com/2026/01/20/us/politics...
1,2026-01-20-19-02-00 +0000,wsj,Supreme Court Voices Skepticism of Hawaii Law ...,https://www.wsj.com/us-news/law/supreme-court-...
2,2026-01-20-19-01-00 +0000,wsj,Precious Metals Gain on Tariff Backlash,https://www.wsj.com/finance/commodities-future...
3,2026-01-20-19-00-42 +0000,nyt,Prosecutors Subpoena Minnesota Democrats as Pa...,https://www.nytimes.com/2026/01/20/us/politics...
4,2026-01-20-18-59-34 +0000,nyt,"Twin Falls, Idaho Has New Residents: 37 Afrika...",https://www.nytimes.com/2026/01/19/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2470/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
78,trump,90
0,greenland,35
2,he,15
76,ice,14
32,new,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
245,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...,184
197,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...,177
169,2026-01-20-12-34-10 +0000,nypost,House Speaker Mike Johnson tells UK Parliament...,https://nypost.com/2026/01/20/us-news/house-sp...,173
241,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/policy/why-an-emb...,172
154,2026-01-20-13-48-20 +0000,nypost,Trump trolls European leaders with wild AI ima...,https://nypost.com/2026/01/20/us-news/trump-tr...,167


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
245,184,2026-01-20-06-08-01 +0000,bbc,Trump says he will '100%' carry out Greenland ...,https://www.bbc.com/news/articles/c4g5345ylk0o...
200,60,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...
123,50,2026-01-20-15-16-37 +0000,nypost,"UK considering its own Australia-style, teen s...",https://nypost.com/2026/01/20/world-news/uk-co...
165,49,2026-01-20-12-56-00 +0000,wsj,Warner Strikes New All-Cash Deal With Netflix,https://www.wsj.com/business/media/warner-stri...
110,46,2026-01-20-15-59-00 +0000,nypost,Don Lemon’s wild war of words with Nicki Minaj...,https://nypost.com/2026/01/20/us-news/don-lemo...
336,40,2026-01-19-19-24-57 +0000,nyt,High-Speed Train Crash in Spain Leaves at Leas...,https://www.nytimes.com/live/2026/01/18/world/...
42,39,2026-01-20-18-03-00 +0000,wsj,A group aligned with Senate Majority Leader Jo...,https://www.wsj.com/politics/policy/gop-spendi...
241,38,2026-01-20-06-39-00 +0000,wsj,President Trump has ramped up pressure on Euro...,https://www.wsj.com/politics/policy/why-an-emb...
312,35,2026-01-19-22-17-41 +0000,nypost,NY Dems in House primary refuse to answer Mamd...,https://nypost.com/2026/01/19/us-news/ny-dems-...
158,34,2026-01-20-13-24-22 +0000,cbc,UN condemns Israel's 'unprecedented' demolitio...,https://www.cbc.ca/news/world/israel-unrwa-bui...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
